In [1]:
# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

In [85]:
# Conectamos con la base de datos:

cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')

# iniciamos el cursor   
mycursor = cnx.cursor()

# Creamos la base de datos si no existe.
try:
    mycursor.execute("CREATE DATABASE IF NOT EXISTS cinemextract")
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)
    
cnx.close()
    

CMySQLCursor: CREATE DATABASE IF NOT EXISTS cinemextra..


In [86]:
cnx.close()

In [87]:
#Nos conectamos a la base de datos que hemos creado

try:
  cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='cinemextract')
  print(cnx)
  
# en caso de que no lo consigas por que hay algún error entonces ...
except mysql.connector.Error as err:

  # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  
  # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  
  # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
  else:
    print(err)


# iniciamos el cursor
   
mycursor = cnx.cursor()   

#Vamos a crear cada una de las tablas:
    
    
try:
  # Tabla Peliculas
  
    mycursor.execute("CREATE TABLE MoviesDataset (tipo_pelicula VARCHAR (45), titulo_pelicula VARCHAR (255), anno_estreno INT,mes_estreno VARCHAR (45),id_pelicula VARCHAR (20), genero_pelicula VARCHAR (45), PRIMARY KEY (id_pelicula))")
  
  # Tabla oscar 
    
    mycursor.execute("CREATE TABLE oscars (id_ceremonia INT,fecha_ceremonia INT,mejor_pelicula VARCHAR (255),mejor_director VARCHAR (100),mejor_actor VARCHAR (100),mejor_actriz VARCHAR (100), PRIMARY KEY (id_ceremonia))")
    
  # Tabla actores
  
    mycursor.execute("CREATE TABLE actores (id_actor INT,nombre_actor VARCHAR (100),anno_nacimiento INT,conocido VARCHAR (45),que_hace VARCHAR (45),premios INT,PRIMARY KEY (id_actor))")
  
  # Tabla detalles peliculas
  
    mycursor.execute("CREATE TABLE detalles_peliculas (id_detalle_peli INT AUTO_INCREMENT, id_pelicula VARCHAR(20),puntuacion_imdb VARCHAR (45),puntuacion_rotten VARCHAR (45),directores VARCHAR (255),guionistas VARCHAR(255),argumento VARCHAR (2000),duracion VARCHAR (45),nombre_pelicula VARCHAR(255), PRIMARY KEY (id_detalle_peli),FOREIGN KEY (id_pelicula) REFERENCES MoviesDataset (id_pelicula))")
  
  # Tabla intermedia actores-peliculas
  
    mycursor.execute("CREATE TABLE int_pelis_actores (id_actor INT,id_pelicula VARCHAR(20),PRIMARY KEY (id_actor,id_pelicula),FOREIGN KEY (id_actor) REFERENCES actores (id_actor),FOREIGN KEY (id_pelicula) REFERENCES MoviesDataset (id_pelicula))")
    
    
    print(mycursor)
except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)
    
    
cnx.close()


CMySQLCursor: CREATE TABLE int_pelis_actores (id_actor..


In [88]:
url ="https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/main/Fase1.csv"
data_pelis =pd.read_csv(url)
# Reemplazar valores nan con un valor predeterminado (por ejemplo, 'N/A')
data_pelis.fillna('N/A', inplace=True)
data_pelis['Mes_estreno'].fillna('N/A', inplace=True)
data_pelis.info()
data_pelis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53008 entries, 0 to 53007
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tipo          53008 non-null  object
 1   Nombre        53008 non-null  object
 2   Anno_estreno  53008 non-null  int64 
 3   Mes_estreno   53008 non-null  object
 4   Id_peli       53008 non-null  object
 5   Genero        53008 non-null  object
dtypes: int64(1), object(5)
memory usage: 2.4+ MB


,Tipo,Nombre,Anno_estreno,Mes_estreno,Id_peli,Genero
0,Movie,Tekken,2010,3,tt0411951,Action
1,Movie,Venus & Vegas,2010,11,tt0423474,Action
2,Movie,The A-Team,2010,6,tt0429493,Action
3,Movie,Segurança Nacional,2010,5,tt0439801,Action
4,Movie,Scott Pilgrim vs. the World,2010,11,tt0446029,Action
...,...,...,...,...,...,...
53003,Short,Helianthus Annuus,2024,N/A,tt27163459,Comedy
53004,Short,This Is Real Love,2024,por estrenar,tt27214982,Comedy
53005,Short,Delicacies,2024,4,tt27614489,Comedy
53006,Short,Niggas in Carlsbad,2024,7,tt27708536,Comedy


In [89]:
lista_pelis=[tuple(i) for i in data_pelis.values]
lista_pelis

[('Movie', 'Tekken', 2010, '3', 'tt0411951', 'Action'),
 ('Movie', 'Venus & Vegas', 2010, '11', 'tt0423474', 'Action'),
 ('Movie', 'The A-Team', 2010, '6', 'tt0429493', 'Action'),
 ('Movie', 'Segurança Nacional', 2010, '5', 'tt0439801', 'Action'),
 ('Movie', 'Scott Pilgrim vs. the World', 2010, '11', 'tt0446029', 'Action'),
 ('Movie', 'Taken by Force', 2010, 'N/A', 'tt0465637', 'Action'),
 ('Movie', 'Guns, Drugs and Dirty Money', 2010, 'N/A', 'tt0464032', 'Action'),
 ('Movie', 'Komaram Puli', 2010, '9', 'tt0464162', 'Action'),
 ('Movie',
  'Prince of Persia: The Sands of Time',
  2010,
  '5',
  'tt0473075',
  'Action'),
 ('Movie', 'Unstoppable', 2010, '12', 'tt0477080', 'Action'),
 ('Movie', 'The Losers', 2010, '4', 'tt0480255', 'Action'),
 ('Movie', 'Clash of the Titans', 2010, '4', 'tt0800320', 'Action'),
 ('Movie', 'Cross the Line', 2010, '11', 'tt0810815', 'Action'),
 ('Movie', 'Gangster Exchange', 2010, '2', 'tt0846004', 'Action'),
 ('Movie', 'Zenitram', 2010, '5', 'tt0861747', 'A

In [90]:
url ="https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/main/Fase1.csv"
data_pelis =pd.read_csv(url)
# Reemplazar valores nan con un valor predeterminado (por ejemplo, 'N/A')
data_pelis.fillna('N/A', inplace=True)
data_pelis['Mes_estreno'].fillna('N/A', inplace=True)
data_pelis_sin_duplicados = data_pelis.drop_duplicates(subset=['Id_peli'])
data_pelis_sin_duplicados.info()
data_pelis_sin_duplicados

lista_pelis=[tuple(i) for i in data_pelis_sin_duplicados.values]
lista_pelis


#Nos conectamos a la base de datos que hemos creado

try:
  cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1',
                              database='cinemextract')
  print(cnx)
  
# en caso de que no lo consigas por que hay algún error entonces ...
except mysql.connector.Error as err:

  # si es un error con la contraseña devuelveme un mensaje de acceso denegado ya que tenemos problemas con la contraseña
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  
  # si el error no tiene que ver con la contraseña, puede ser porque la base de datos no exista, devuelveme un mensaje de que la base de datos no existe
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  
  # si no es por ninguno de los errores anteriores, printeame cual es el error que estoy teniendo en mi conexión
  else:
    print(err)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12482 entries, 0 to 53007
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tipo          12482 non-null  object
 1   Nombre        12482 non-null  object
 2   Anno_estreno  12482 non-null  int64 
 3   Mes_estreno   12482 non-null  object
 4   Id_peli       12482 non-null  object
 5   Genero        12482 non-null  object
dtypes: int64(1), object(5)
memory usage: 682.6+ KB


In [91]:
# iniciamos el cursor
   
mycursor = cnx.cursor()

sql = "INSERT INTO moviesdataset (tipo_pelicula, titulo_pelicula, anno_estreno, mes_estreno,id_pelicula,genero_pelicula) VALUES (%s, %s, %s, %s,%s,%s)" 


                        
                        #peliculas el nombre del archivo correspondiente
try:
    mycursor.executemany(sql, lista_pelis)
    cnx.commit()
    print(mycursor.rowcount,"registros insertados")
except mysql.connector.Error as err:
    print("Ha habido un error en la inserción")
    print(err)

12482 registros insertados


In [97]:
url ="https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/main/Fase2.csv"
data_detalles =pd.read_csv(url)
# Reemplazar valores nan con un valor predeterminado (por ejemplo, 'N/A')
data_detalles.fillna('N/A', inplace=True)
data_detalles['Tomatometer'].fillna('N/A', inplace=True)
#data_detalles_sin_duplicados = data_detalles.drop_duplicates(subset=['Id_peli'])

data_detalles.info()
data_detalles

lista_detalles=[tuple(i) for i in data_detalles.values]
lista_detalles



# iniciamos el cursor
   
mycursor = cnx.cursor()

sql = "INSERT INTO detalles_peliculas (id_pelicula,puntuacion_imdb,puntuacion_rotten,directores,guionistas,argumento,duracion,nombre_pelicula) VALUES (%s, %s, %s,%s,%s, %s,%s,%s)" 


                        
#premios oscar

try:
    mycursor.executemany(sql, lista_detalles)
    cnx.commit()
    print(mycursor.rowcount,"registros insertados")
except mysql.connector.Error as err:
    print("Ha habido un error en la inserción")
    print(err)
    
    
#NO FUNCIONA PORQUE NINGUNA DE LAS PELICULAS QUE ESTAN EN LA LISTA EXISTE EN EL LISTADO DE LAS PELICULAS.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Id_peli      5 non-null      object
 1   Punt_IMDB    5 non-null      object
 2   Tomatometer  5 non-null      object
 3   Direccion    5 non-null      object
 4   Guionistas   5 non-null      object
 5   Argumento    5 non-null      object
 6   Duracion     5 non-null      object
 7   Nombre       5 non-null      object
dtypes: object(8)
memory usage: 448.0+ bytes
Ha habido un error en la inserción
1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`cinemextract`.`detalles_peliculas`, CONSTRAINT `detalles_peliculas_ibfk_1` FOREIGN KEY (`id_pelicula`) REFERENCES `moviesdataset` (`id_pelicula`))
